In [1]:
import jax
import jax.numpy as jnp
import haiku as hk
import optax
import sys
from time import sleep
import json
import pexpect
import re
import timeit


In [2]:
HOLPATH = "/home/sean/Documents/PhD/HOL4/HOL/bin/hol --maxheap=256"
TARGET_THEORIES = ["pred_set"] #["arithmetic"]#["list"] #["rich_list"] # ["integer"] #["arithmetic"] # ["rich_list"] #["pred_set"]

In [3]:
with open("typed_database.json") as f:
    database = json.load(f)

In [4]:
#goals = [value[4] for key, value in database.items() if value[3] == "thm" and value[0] in TARGET_THEORIES]

In [5]:
#goals

In [6]:
def get_polish(raw_goal):
        goal = construct_goal(raw_goal)
        process.sendline(goal.encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("val _ = set_term_printer (HOLPP.add_string o pt);".encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("top_goals();".encode("utf-8"))
        process.expect("val it =")
        process.expect([": goal list", ":\r\n +goal list"])

        polished_raw = process.before.decode("utf-8")
        polished_subgoals = re.sub("“|”","\"", polished_raw)
        polished_subgoals = re.sub("\r\n +"," ", polished_subgoals)

        # print("content:{}".format(subgoals))
        # exit()
        pd = eval(polished_subgoals)
        
        process.expect("\r\n>")
        process.sendline("drop();".encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("val _ = set_term_printer default_pt;".encode("utf-8"))
        process.expect("\r\n>")

        data = [{"polished":{"assumptions": e[0][0], "goal":e[0][1]},
                 "plain":{"assumptions": e[1][0], "goal":e[1][1]}}
                for e in zip(pd, [([], raw_goal)])]
        return data # list(zip(pd, [([], raw_goal)]))f

def construct_goal(goal):
    s = "g " + "`" + goal + "`;"
    return s

In [7]:
process = pexpect.spawn(HOLPATH)

theories = ["listTheory", "bossLib"]

print("Importing theories...")
process.sendline("val _ = HOL_Interactive.toggle_quietdec();".encode("utf-8"))
for i in theories:
    process.sendline("open {};".format(i).encode("utf-8"))

Importing theories...


In [8]:

# # remove built-in simp lemmas
# print("Removing simp lemmas...")
# process.sendline("delsimps [\"HD\", \"EL_restricted\", \"EL_simp_restricted\"];")
# #process.sendline("delsimps {};".format(dels))
# #process.sendline("delsimps {};".format(dels2))
# # process.sendline("delsimps {};".format(dels3))
# sleep(4)
# # load utils
# print("Loading modules...")
# process.sendline("use \"helper.sml\";")
# sleep(5)
# # process.sendline("val _ = load \"Timeout\";")
# print("Configuration done.")
# process.expect('\r\n>')
# # process.readline()
# process.sendline("val _ = HOL_Interactive.toggle_quietdec();".encode("utf-8"))

# # consumes hol4 head
# process.expect('\r\n>')
# #print (process.read())
# # setup the goal
# goal = goals[0]
                      
#  # a pair of polished goal list and original goal list
# fringe = get_polish(goal)

# #scripts = []
# goal = construct_goal(goal)
# process.sendline(goal.encode("utf-8"))        
# print("Initialization done. Main goal is:\n{}.".format(goal))


In [9]:
# print (fringe)

In [10]:
tactic_zero_path = "/home/sean/Documents/PhD/git/repo/PhD/tacticzero/holgym/"


In [11]:
test_history = [{'content': [{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE l = [] ⇔ l = []'}}], 'parent': None, 'goal': None, 'by_tactic': '', 'reward': None}, {'content': [{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Clist$NIL Clist$NIL @ @ Cmin$= Clist$NIL Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE [] = [] ⇔ [] = []'}}, {'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'assumptions': ['REVERSE l = [] ⇔ l = []'], 'goal': '∀h. REVERSE (h::l) = [] ⇔ h::l = []'}}], 'parent': 0, 'goal': 0, 'by_tactic': 'Induct_on `l`', 'reward': 0.1}, {'content': [{'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'assumptions': ['REVERSE l = [] ⇔ l = []'], 'goal': '∀h. REVERSE (h::l) = [] ⇔ h::l = []'}}], 'parent': 1, 'goal': 0, 'by_tactic': 'fs[EL_simp_restricted, LENGTH_ZIP, SUM_eq_0, LENGTH_EQ_NUM, FILTER_NEQ_NIL]', 'reward': 0.2}, {'content': [], 'parent': 2, 'goal': 0, 'by_tactic': 'fs[APPEND_LENGTH_EQ, LIST_REL_rules, FILTER_NEQ_NIL, REVERSE_11, LENGTH_REVERSE]', 'reward': 5}]



In [12]:
def gather_encoded_content(history, encoder):
    # figure out why this is slower than tests
    # figured out: remember to do strip().split()
    fringe_sizes = []
    contexts = []
    reverted = []
    for i in history:
        c = i["content"]
        contexts.extend(c)
        fringe_sizes.append(len(c))
    for e in contexts:
        g = revert_with_polish(e)
        reverted.append(g.strip().split())
    out = []
    sizes = []
    for goal in reverted:
        out_, sizes_ = batch_encoder.encode([goal])
        out.append(torch.cat(out_.split(1), dim=2).squeeze(0))
        sizes.append(sizes_)
        
    # s1 = timeit.default_timer()
    #out, sizes = batch_encoder.encode(reverted)
    
    representations = out
    # merge two hidden variables
    #representations = torch.cat(out.split(1), dim=2).squeeze(0)
    # print(representations.shape)
    # s2 = timeit.default_timer()    
    # print(s2-s1)

    return representations, contexts, fringe_sizes

In [13]:
ARG_LEN = 5 
EXCLUDED_THEORIES = ["min"] #["min", "bool"]
def replay_known_proof(known_history, arg_len=ARG_LEN):
    print("\nReplaying a known proof of {} ...".format(known_history[0]["content"][0]["plain"]["goal"]))
    start = timeit.default_timer()

    fringe_pool = []
    tac_pool = []
    arg_pool = []
    reward_pool = []
    reward_print = []
    action_pool = []
    steps = 0
    flag = True
    tac_print = []

    induct_arg = []
    proved = 0
    iteration_rewards = []

    # compute the encoded fact pool at the beginning of each episode
    polished_goal = known_history[0]["content"][0]["polished"]["goal"]
    allowed_theories = parse_theory(polished_goal)
    allowed_theories = [t for t in allowed_theories if t not in EXCLUDED_THEORIES]
    
    try:
        allowed_arguments_ids = []
        candidate_args = []
        goal_theory = database[polished_goal][0]
        for i,t in enumerate(database):
            if database[t][0] in allowed_theories and (database[t][0] != goal_theory or int(database[t][2]) < int(database[polished_goal][2])):
                allowed_arguments_ids.append(i)
                candidate_args.append(t)

    except:
        allowed_arguments_ids = []
        candidate_args = []
        for i,t in enumerate(database):
            if database[t][0] in allowed_theories:
                allowed_arguments_ids.append(i)
                candidate_args.append(t)
        print("Theorem not found in database.")

    #encoded_fact_pool = torch.index_select(encoded_database, 0, torch.tensor(allowed_arguments_ids, device=device))

    # print("Facts: {}+{}".format(num_init_facts, len(split_fact_pool)-num_init_facts))
    print("Facts: {}".format(len(allowed_arguments_ids)))

    for t in range(10):
        true_resulting_fringe = known_history[t+1]

        # gather all the goals in the history
       # try:
        print (known_history)
        #known_history[t+1]
        representations, context_set, fringe_sizes = gather_encoded_content(known_history, batch_encoder)
#         except::
#             env.close()
#             print("Skipping current game due to encoding error ...")
#             print("Restarting environment ...")
#             print(env.goal)                
#             env = HolEnv(env.goal)
#             flag = False
#             break
            
        return representations, context_set, fringe_sizes
        #representations = representations.to(device)
        context_scores = context_net(representations)
        contexts_by_fringe, scores_by_fringe = split_by_fringe(context_set, context_scores, fringe_sizes)
        fringe_scores = []
        for s in scores_by_fringe:
            # fringe_score = torch.prod(s) # TODO: make it sum
            fringe_score = torch.sum(s) # TODO: make it sum
            fringe_scores.append(fringe_score)
        fringe_scores = torch.stack(fringe_scores)
        fringe_probs = F.softmax(fringe_scores, dim=0)
        fringe_m = Categorical(fringe_probs)
        # fringe = fringe_m.sample()
        
        true_fringe = torch.tensor([true_resulting_fringe["parent"]])
        true_fringe = true_fringe.to(device)
        
        fringe_pool.append(fringe_m.log_prob(true_fringe))

        # take the first context in the chosen fringe for now

        target_context = contexts_by_fringe[true_fringe][0]
        target_goal = target_context["polished"]["goal"]
        target_representation = representations[context_set.index(target_context)]
        # print(target_representation.shape)
        # exit()

        # size: (1, max_contexts, max_assumptions+1, max_len)
        tac_input = target_representation.unsqueeze(0)
        tac_input = tac_input.to(device)

        # compute scores of tactics
        tac_probs = tac_net(tac_input)
        # print(tac_probs)
        tac_m = Categorical(tac_probs)

        true_tactic_text = true_resulting_fringe["by_tactic"]

        # true_tactic_text = "Induct_on `ll`"
        tac_args = re.findall(r'(.*?)\[(.*?)\]', true_tactic_text)
        tac_term = re.findall(r'(.*?) `(.*?)`', true_tactic_text)     
        tac_arg = re.findall(r'(.*?) (.*)', true_tactic_text)

        if tac_args:
            true_tac_text = tac_args[0][0]
            true_args_text = tac_args[0][1].split(", ")
        elif tac_term: # order matters # TODO: make it irrelavant
            true_tac_text = tac_term[0][0]
            true_args_text = tac_term[0][1]
        elif tac_arg: # order matters because tac_arg could match () ``
            true_tac_text = tac_arg[0][0]
            true_args_text = tac_arg[0][1]
        else:
            true_tac_text = true_tactic_text
        
        true_tac = torch.tensor([tactic_pool.index(true_tac_text)])
        true_tac = true_tac.to(device)
        # tac = tac_m.sample()
        # log directly the log probability
        tac_pool.append(tac_m.log_prob(true_tac))
        
        action_pool.append(tactic_pool[true_tac])
        
        tac_print.append(tac_probs.detach())
        # print(len(fact_pool[0].strip().split()))
        # exit()

        tac_tensor = true_tac.to(device)

        assert tactic_pool[true_tac.item()] == true_tac_text
        
        if tactic_pool[true_tac] in no_arg_tactic:
            tactic = tactic_pool[true_tac]
            arg_probs = []
            arg_probs.append(torch.tensor(0))
            arg_pool.append(arg_probs)
        elif tactic_pool[true_tac] == "Induct_on":
            arg_probs = []
            candidates = []
            # input = torch.cat([target_representation, tac_tensor], dim=1)
            tokens = target_goal.split()
            tokens = list(dict.fromkeys(tokens))
            tokens = [[t] for t in tokens if t[0] == "V"]
            if tokens:
                # concatenate target_representation to token
                # use seq2seq to compute the representation of a token
                # also we don't need to split an element in tokens because they are singletons
                # but we need to make it a list containing a singleton list, i.e., [['Vl']]

                token_representations, _ = batch_encoder.encode(tokens)
                # reshaping
                encoded_tokens = torch.cat(token_representations.split(1), dim=2).squeeze(0)
                target_representation_list = [target_representation.unsqueeze(0) for _ in tokens]

                target_representations = torch.cat(target_representation_list)
                # size: (len(tokens), 512)
                candidates = torch.cat([encoded_tokens, target_representations], dim=1)
                candidates = candidates.to(device)

                # concat = [torch.cat([torch.tensor([input_vocab.stoi[i] for _ in range(256)], dtype=torch.float), target_representation]) for i in tokens]

                # candidates = torch.stack(concat)
                # candidates = candidates.to(device)

                scores = term_net(candidates, tac_tensor)
                term_probs = F.softmax(scores, dim=0)
                try:
                    term_m = Categorical(term_probs.squeeze(1))
                except:
                    print("probs: {}".format(term_probs))                                          
                    print("candidates: {}".format(candidates.shape))
                    print("scores: {}".format(scores))
                    print("tokens: {}".format(tokens))
                    exit()
                    
                true_term = torch.tensor([tokens.index(["V" + true_args_text])])
                true_term = true_term.to(device)
                # term = term_m.sample()
                
                arg_probs.append(term_m.log_prob(true_term))
                induct_arg.append(tokens[true_term])
                tm = tokens[true_term.item()][0][1:] # remove headers, e.g., "V" / "C" / ...
                
                assert tm == true_args_text
                
                arg_pool.append(arg_probs)
                if tm:
                    tactic = "Induct_on `{}`".format(tm)
                else:
                    # only to raise an error
                    tactic = "Induct_on"
            else:
                arg_probs.append(torch.tensor(0))
                induct_arg.append("No variables")
                arg_pool.append(arg_probs)
                tactic = "Induct_on"
        else:
            hidden0 = hidden1 = target_representation.unsqueeze(0).unsqueeze(0)

            hidden0 = hidden0.to(device)
            hidden1 = hidden1.to(device)

            hidden = (hidden0, hidden1)

            # concatenate the candidates with hidden states.

            hc = torch.cat([hidden0.squeeze(), hidden1.squeeze()])
            hiddenl = [hc.unsqueeze(0) for _ in allowed_arguments_ids]

            hiddenl = torch.cat(hiddenl)

            # size: (len(fact_pool), 512)
            candidates = torch.cat([encoded_fact_pool, hiddenl], dim=1)
            candidates = candidates.to(device)

            input = tac_tensor
            # run it once before predicting the first argument
            hidden, _ = arg_net(input, candidates, hidden)

            # the indices of chosen args
            arg_step = []
            arg_step_probs = []
            # print("True args text: {}".format(true_args_text))
            if tactic_pool[true_tac] in thm_tactic:
                arg_len = 1
            else:
                arg_len = ARG_LEN

            for i in range(arg_len):
                hidden, scores = arg_net(input, candidates, hidden)
                arg_probs = F.softmax(scores, dim=0)
                arg_m = Categorical(arg_probs.squeeze(1))
                if isinstance(true_args_text, list):
                    try:
                        name_parser = true_args_text[i].split(".")
                    except:
                        print(i)
                        print(true_args_text)
                        print(known_history)
                        exit()
                    theory_name = name_parser[0][:-6] # get rid of the "Theory" substring
                    theorem_name = name_parser[1]
                    true_arg_exp = reverse_database[(theory_name, theorem_name)]
                else:
                    name_parser = true_args_text.split(".")
                    theory_name = name_parser[0][:-6] # get rid of the "Theory" substring
                    theorem_name = name_parser[1]
                    true_arg_exp = reverse_database[(theory_name, theorem_name)]    
                true_arg = torch.tensor(candidate_args.index(true_arg_exp))
                true_arg = true_arg.to(device)
                # arg = arg_m.sample()
                
                arg_step.append(true_arg)
                arg_step_probs.append(arg_m.log_prob(true_arg))

                # hiddenc0 = hidden[0].squeeze().repeat(1, 1, 1)
                # hiddenc1 = hidden[1].squeeze().repeat(1, 1, 1)

                # encoded chosen argument
                input = encoded_fact_pool[true_arg.item()].unsqueeze(0).unsqueeze(0)
                # print(input.shape)

                # renew candidates                
                hc = torch.cat([hidden0.squeeze(), hidden1.squeeze()])
                hiddenl = [hc.unsqueeze(0) for _ in allowed_arguments_ids]

                hiddenl = torch.cat(hiddenl)

                # size: (len(fact_pool), 512)
                candidates = torch.cat([encoded_fact_pool, hiddenl], dim=1)
                candidates = candidates.to(device)

            arg_pool.append(arg_step_probs)

            # tac = tactic_pool[true_tac]
            # arg = [candidate_args[i] for i in arg_step]

            # tactic = env.assemble_tactic(tac, arg)
            # assert tactic == true_tactic_text

        action = (true_fringe.item(), 0, true_tactic_text)
        try:
            reward = true_resulting_fringe["reward"]
            done = t+2 == len(known_history)

        except:
            print("Step exception raised.")
            # print("Fringe: {}".format(env.history))
            print("Handling: {}".format(env.handling))
            print("Using: {}".format(env.using))
            # try again
            # counter = env.counter
            frequency = env.frequency
            env.close()
            print("Aborting current game ...")
            print("Restarting environment ...")
            print(env.goal)
            env = HolEnv(env.goal)
            flag = False
            break

        # state_pool.append(state)
        reward_print.append(reward)
        # reward_pool.append(reward+trade_off*entropy)
        reward_pool.append(reward)

        # pg = ng

        steps += 1
        total_reward = float(np.sum(reward_print))
        if done == True:
            print("Proved in {} steps.".format(t+1))
            print("Rewards: {}".format(reward_print))
            # print("Tactics: {}".format(action_pool))
            # # print("Mean reward: {}".format(np.mean(reward_pool)))
            # print("Total: {}".format(total_reward))
            # print("Proof trace: {}".format(extract_proof(known_history)))
            # try:
            #     print("Proof script: {}".format(reconstruct_proof(known_history)))
            # except:
            #     print("Proof check failed with error.")
            #     proof_check_failure.append(known_history)

            # exit()
            proved += 1
            # traces.append(env.history)
            iteration_rewards.append(total_reward)
            break

    stop = timeit.default_timer()
    print('Time: {}  '.format(stop - start))


    # Update policy
    if True:
        # Discount reward
        running_add = 0
        for i in reversed(range(steps)):
            if reward_pool[i] == 0:
                running_add = 0
            else:
                running_add = running_add * gamma + reward_pool[i]
                reward_pool[i] = running_add

        optimizer_context.zero_grad()
        optimizer_tac.zero_grad()
        optimizer_arg.zero_grad()
        optimizer_term.zero_grad()

        for i in range(steps):
            # size : (1,1,4,128)
            total_loss = 0

            # state = state_pool[i]
            reward = reward_pool[i]

            fringe_loss = -fringe_pool[i] * (reward)
            arg_loss = -torch.sum(torch.stack(arg_pool[i])) * (reward)

            tac_loss = -tac_pool[i] * (reward)

            # entropy = fringe_pool[i] + torch.sum(torch.stack(arg_pool[i])) + tac_pool[i]

            # loss = fringe_loss + tac_loss + arg_loss + trade_off*entropy
            loss = fringe_loss + tac_loss + arg_loss
            # total_loss += loss

            loss.backward()

        # total_loss.backward()

        # optimizer.step()

        optimizer_context.step()
        optimizer_tac.step()
        optimizer_arg.step()
        # optimizer_term.step()

    prf = torch.mean(torch.stack(tac_print), 0)
    print("Preferences: {}\n".format(prf))

    # state_pool = []
    fringe_pool = []
    tac_pool = []
    arg_pool = []
    action_pool = []
    reward_pool = []
    reward_print = []
    steps = 0
    flag = True
    
    
def parse_theory(pg):
    theories = re.findall(r'C\$(\w+)\$ ', pg)
    theories = set(theories)
    for th in EXCLUDED_THEORIES:
        theories.discard(th)
    return list(theories)

def revert_with_polish(context):
    target = context["polished"]
    assumptions = target["assumptions"]
    goal = target["goal"]
    for i in reversed(assumptions): 
        goal = "@ @ Dmin$==> {} {}".format(i, goal)
    return goal 

def split_by_fringe(goal_set, goal_scores, fringe_sizes):
    # group the scores by fringe
    fs = []
    gs = []
    counter = 0
    for i in fringe_sizes:
        end = counter + i
        fs.append(goal_scores[counter:end])
        gs.append(goal_set[counter:end])
        counter = end
    return gs, fs

In [14]:
import os
import argparse
import logging
import timeit
import torch

import seq2seq
#from seq2seq.evaluator import BatchPredictor
#from seq2sebq.util.checkpoint import Checkpoint

from batch_predictor import BatchPredictor
from checkpoint import Checkpoint

#checkpoint_path = "models/2021_02_21_15_46_04" # 98% accuracy model, up to probability theory
checkpoint_path = "models/2020_04_26_20_11_28" # 95% accuracy model, core theories + integer + sorting

#checkpoint_path = "models/2021_02_22_16_07_03" # 97-98% accuracy model, up to and include probability theory
#checkpoint_path = "models/2020_09_24_23_38_06" # 98% accuracy model, core theories + integer + sorting | separate theory tokens


# logging.info("loading checkpoint from {}".format(checkpoint_path))

checkpoint = Checkpoint.load(checkpoint_path)
seq2seq = checkpoint.model
input_vocab = checkpoint.input_vocab
output_vocab = checkpoint.output_vocab

batch_encoder = BatchPredictor(seq2seq, input_vocab, output_vocab)


/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'seq2seq.models.EncoderRNN.EncoderRNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/sean/Documents/venvs/jax/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to reve

In [15]:
jax_reps, context_set, fringe_sizes = replay_known_proof(test_history)


Replaying a known proof of REVERSE l = [] ⇔ l = [] ...
Theorem not found in database.
Facts: 0
[{'content': [{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE l = [] ⇔ l = []'}}], 'parent': None, 'goal': None, 'by_tactic': '', 'reward': None}, {'content': [{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Clist$NIL Clist$NIL @ @ Cmin$= Clist$NIL Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE [] = [] ⇔ [] = []'}}, {'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'assumptions': ['REVERSE l = [] ⇔ l = []'], 'goal': '∀h. REVERSE (h::l) = [] ⇔ h::l = []'}}], 'parent': 0, 'goal': 0, 'by_tactic': 'Induct_on `l`', 'reward': 0.1}, {'c

In [16]:
print (context_set, fringe_sizes)

[{'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE l = [] ⇔ l = []'}}, {'polished': {'assumptions': [], 'goal': '@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Clist$NIL Clist$NIL @ @ Cmin$= Clist$NIL Clist$NIL'}, 'plain': {'assumptions': [], 'goal': 'REVERSE [] = [] ⇔ [] = []'}}, {'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'assumptions': ['REVERSE l = [] ⇔ l = []'], 'goal': '∀h. REVERSE (h::l) = [] ⇔ h::l = []'}}, {'polished': {'assumptions': ['@ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE Vl Clist$NIL @ @ Cmin$= Vl Clist$NIL'], 'goal': '@ Cbool$! | Vh @ @ Cmin$= @ @ Cmin$= @ Clist$REVERSE @ @ Clist$CONS Vh Vl Clist$NIL @ @ Cmin$= @ @ Clist$CONS Vh Vl Clist$NIL'}, 'plain': {'ass

In [17]:
from policy_networks import *
import policy_networks

In [18]:
init_context, apply_context = hk.transform(policy_networks._context_forward)
rng_key = random.PRNGKey(100)

#convert to jax
jax_reps = jnp.stack([jnp.array(jax_reps[i][0]) for i in range(len(jax_reps))])


context_params = init_context(rng_key, jax_reps)
apply_context = jax.jit(apply_context)


In [19]:
# context_scores = apply_context(context_params, rng_key, jax_reps)
# print (context_scores)

In [20]:

#function to give the log probability of pi(f | s) so gradient can be computed directly
#also returns sampled index and contexts to determine goal to give tactic network
def sample_fringe(context_params, context_net, rng_key, jax_reps, context_set, fringe_sizes):
    context_scores = context_net(context_params, rng_key, jax_reps)
    contexts_by_fringe, scores_by_fringe = split_by_fringe(context_set, context_scores, fringe_sizes)
    fringe_scores = []
    for s in scores_by_fringe:
        fringe_score = jnp.sum(s)
        fringe_scores.append(fringe_score)
    #TODO some fringes can be empty, but still give value 0 which assigns nonzero probability?
    fringe_scores = jnp.stack(fringe_scores)
    fringe_probs = jax.nn.softmax(fringe_scores)

    #samples, gives an index (looks like it does gumbel softmax under the hood to keep differentiability?)
    sampled_idx = random.categorical(rng_key,fringe_probs)

    prob = fringe_probs[sampled_idx]
    log_prob = jnp.log(prob)
    return log_prob, (sampled_idx, contexts_by_fringe)
                                                           
grad_log_context, (fringe_idx, contexts_by_fringe) = jax.grad(sample_fringe, has_aux=True)(context_params, apply_context, rng_key, jax_reps, context_set, fringe_sizes)
print (grad_log_context, fringe_idx, contexts_by_fringe)

{'context_policy/~/linear': {'b': DeviceArray([-1.07554905e-02, -1.54109523e-02,  0.00000000e+00,
              0.00000000e+00,  0.00000000e+00,  8.85673426e-03,
              0.00000000e+00, -1.18663358e-02, -7.72345625e-03,
              0.00000000e+00,  9.60543007e-03, -2.04063312e-04,
              1.32207945e-03,  0.00000000e+00, -1.41528305e-02,
              1.16932672e-02, -1.79509595e-02,  0.00000000e+00,
              0.00000000e+00,  1.49002681e-02,  0.00000000e+00,
              0.00000000e+00, -9.41692851e-04, -1.20092742e-02,
              5.15160523e-03, -1.57083152e-04,  1.06401350e-02,
             -8.68311524e-03,  0.00000000e+00,  2.36719032e-03,
              0.00000000e+00,  7.62443151e-03, -8.41719657e-03,
             -3.98649648e-03,  0.00000000e+00,  1.52461766e-03,
              3.47098918e-04,  0.00000000e+00,  0.00000000e+00,
              0.00000000e+00,  0.00000000e+00,  7.20718317e-03,
              6.49480056e-03,  0.00000000e+00,  0.00000000e+00,
      

In [21]:
target_context = contexts_by_fringe[fringe_idx][0]
target_goal = target_context["polished"]["goal"]
target_representation = jax_reps[context_set.index(target_context)]
# print(target_representation.shape)
# exit()

# size: (1, max_contexts, max_assumptions+1, max_len)
# tac_input = target_representation.unsqueeze(0)
# tac_input = tac_input.to(device)

# # compute scores of tactics
# tac_probs = tac_net(tac_input)
# # print(tac_probs)
# tac_m = Categorical(tac_probs)
# tac = tac_m.sample()

# print (target_context)
# print (target_goal)
#print (target_representation)



In [22]:
MORE_TACTICS = True
if not MORE_TACTICS:
    thms_tactic = ["simp", "fs", "metis_tac"]
    thm_tactic = ["irule"]
    term_tactic = ["Induct_on"]
    no_arg_tactic = ["strip_tac"]
else:
    thms_tactic = ["simp", "fs", "metis_tac", "rw"]
    thm_tactic = ["irule", "drule"]
    term_tactic = ["Induct_on"]
    no_arg_tactic = ["strip_tac", "EQ_TAC"]
    
    # thms_tactic = ["simp", "fs", "metis_tac", "rw"]
    # thm_tactic = [] #["irule", "drule"] 
    # term_tactic = ["Induct_on"]
    # no_arg_tactic = ["strip_tac", "EQ_TAC", "simp[]", "rw[]", "metis_tac[]", "fs[]"]

tactic_pool = thms_tactic + thm_tactic + term_tactic + no_arg_tactic
print (len(tactic_pool))

9


In [23]:
init_tac, apply_tac = hk.transform(policy_networks._tac_forward)

tactic_params = init_tac(rng_key, jnp.expand_dims(target_representation,0), len(tactic_pool))
apply_tac = partial(jax.jit, static_argnums=3)(apply_tac)


In [24]:
#takes a goal encoding and samples tactic from network, and returns log prob for gradient 
def sample_tactic(tactic_params, tac_net, rng_key, goal_endcoding, action_size=len(tactic_pool)):
    tac_probs = tac_net(tactic_params, rng_key, goal_endcoding, action_size)[0]
    tac_idx = random.categorical(rng_key, tac_probs)
    log_prob = jnp.log(tac_probs[tac_idx])
    return log_prob, tac_idx

grad_log_tac, tac_idx = jax.grad(sample_tactic, has_aux=True)(tactic_params, apply_tac, rng_key, jnp.expand_dims(target_representation,0), len(tactic_pool))

sampled_tac = tactic_pool[tac_idx]

In [25]:
#for testing

#if tactic requires no argument
if sampled_tac in no_arg_tactic:
    full_tactic = tactic_pool[tac]
    
#Induct_on case; use term policy to find which term to induct on 
elif sampled_tac in term_tactic:
    goal_tokens = target_goal.split()
    term_tokens = [[t] for t in set(goal_tokens) if t[0] == "V"]
    #add conditional if tokens is empty 
    
    

In [26]:
goal_tokens = target_goal.split()
#goal_tokens = dict.fromkeys(goal_tokens): set should be better?
term_tokens = [[t] for t in set(goal_tokens) if t[0] == "V"]

print (term_tokens)

[['Vh'], ['Vl']]


In [27]:
#now want encodings for terms from AE

term_reps = []

for term in term_tokens:
    term_rep, _ = batch_encoder.encode([term])
    #output is bidirectional so concat vectors
    term_reps.append(torch.cat(term_rep.split(1), dim=2).squeeze(0))

# convert to jax
term_reps = jnp.stack([jnp.array(term_reps[i][0]) for i in range(len(term_reps))])
term_reps

DeviceArray([[ 2.50416473e-02, -3.20719659e-01, -3.78435142e-02,
              -2.00626925e-01, -2.83700794e-01, -1.41217113e-01,
              -5.73794916e-02, -2.04383641e-01,  1.35017827e-01,
               5.38408935e-01, -5.26751801e-02, -3.93549576e-02,
              -1.25084887e-03, -9.41106603e-02,  2.13240564e-01,
               1.38817221e-01,  4.80480045e-01, -2.60695696e-01,
               2.19320253e-01,  9.91149247e-02,  7.66791701e-02,
               1.16289407e-01, -2.48965174e-01,  2.29843348e-01,
              -1.02492506e-02, -2.50420809e-01, -1.25457913e-01,
              -2.00330794e-01, -3.27185124e-01, -9.72870141e-02,
              -1.96665674e-01, -3.47513482e-02,  1.89577311e-01,
              -1.85463637e-01, -8.75676423e-02,  2.57773072e-01,
              -5.82295377e-03,  2.02088043e-01, -2.03279145e-02,
              -1.02889445e-02, -1.85730755e-01, -5.31250238e-02,
               2.14131624e-02, -4.99357432e-02,  3.34141999e-02,
              -1.38412938

In [28]:
# now want inputs to term_net to be target_representation (i.e. goal) concatenated with terms
# models the policies conditional dependence of the term given the goal

#stack goal representation for each token
goal_stack = jnp.concatenate([jnp.expand_dims(target_representation,0) for _ in term_tokens])

#concat with term encodings to give candidate matrix
candidates = jnp.concatenate([goal_stack, term_reps], 1)

print (candidates.shape)

(2, 512)


In [29]:
# init_term, apply_term = hk.transform(policy_networks._term_forward)

# #TODO why do we need the tactic? The use of term net implicitly assumes we are using induct_on, and every time this is called the tactic index will be the same?

# term_params = init_term(rng_key, candidates, tac_idx, len(tactic_pool), candidates.shape[1])
# apply_term = partial(jax.jit, static_argnums=(4,5))(apply_term)


In [30]:
# #takes a goal encoding and samples tactic from network, and returns log prob for gradient 
# def sample_term(term_params, term_net, rng_key, candidates, tactic,TAC_SIZE, MAX_LEN):
#     term_scores = term_net(term_params, rng_key, candidates, tactic, TAC_SIZE, MAX_LEN)
#     term_scores = jnp.ravel(term_scores)
#     term_probs = jax.nn.softmax(term_scores)
#     term_idx = random.categorical(rng_key, term_probs)
#     log_prob = jnp.log(term_probs[term_idx])
#     return log_prob, term_idx

# grad_log_term, tac_term = jax.grad(sample_term, has_aux=True)(term_params, apply_term, rng_key, candidates, tac_idx, len(tactic_pool), candidates.shape[1])

# print (grad_log_term, tac_term)

In [31]:
#term policy implementation without tactic as an argument (since it will always be Induct_on)
#policy will now learn a mapping for V(term | goal)

init_term, apply_term = hk.transform(policy_networks._term_no_tac_forward)
term_params = init_term(rng_key, candidates)
apply_term = jax.jit(apply_term)


In [32]:
def sample_term(term_params, term_net, rng_key, candidates):
    term_scores = term_net(term_params, rng_key, candidates)
    term_scores = jnp.ravel(term_scores)
    term_probs = jax.nn.softmax(term_scores)
    term_idx = random.categorical(rng_key, term_probs)
    log_prob = jnp.log(term_probs[term_idx])
    return log_prob, term_idx

grad_log_term, term_idx = jax.grad(sample_term, has_aux=True)(term_params, apply_term, rng_key, candidates)#, tac_idx, len(tactic_pool), candidates.shape[1])

#print (grad_log_term, term_idx)

In [33]:
sampled_term = term_tokens[term_idx]

tm = sampled_term[0][1:] # remove headers, e.g., "V" / "C" / ...

if tm:
    tactic = "Induct_on `{}`".format(tm)
else:
    # only to raise an error
    tactic = "Induct_on"

In [34]:
#get candidate arguments

with open("polished_def_dict.json") as f:
    defs = json.load(f)
    # print(list(dictionary.keys()))
# with open("polished_thm_dict_sorted.json") as f:
#     pthms = json.load(f)
fact_pool = list(defs.keys())
fact_pool

['@ @ Cbool$/\\ @ Cbool$! | Vl2 @ @ Cmin$= @ Clist$ZIP @ @ Cpair$, Clist$NIL Vl2 Clist$NIL @ @ Cbool$/\\ @ Cbool$! | Vl1 @ @ Cmin$= @ Clist$ZIP @ @ Cpair$, Vl1 Clist$NIL Clist$NIL @ Cbool$! | Vx1 @ Cbool$! | Vl1 @ Cbool$! | Vx2 @ Cbool$! | Vl2 @ @ Cmin$= @ Clist$ZIP @ @ Cpair$, @ @ Clist$CONS Vx1 Vl1 @ @ Clist$CONS Vx2 Vl2 @ @ Clist$CONS @ @ Cpair$, Vx1 Vx2 @ Clist$ZIP @ @ Cpair$, Vl1 Vl2',
 '@ @ Cbool$/\\ @ @ Cmin$= @ Clist$UNZIP Clist$NIL @ @ Cpair$, Clist$NIL Clist$NIL @ Cbool$! | Vx @ Cbool$! | Vl @ @ Cmin$= @ Clist$UNZIP @ @ Clist$CONS Vx Vl @ @ Cpair$, @ @ Clist$CONS @ Cpair$FST Vx @ Cpair$FST @ Clist$UNZIP Vl @ @ Clist$CONS @ Cpair$SND Vx @ Cpair$SND @ Clist$UNZIP Vl',
 '@ Cbool$! | Ve @ Cbool$! | VL @ @ Cmin$= @ @ Clist$UNIQUE Ve VL @ Cbool$? | VL1 @ Cbool$? | VL2 @ @ Cbool$/\\ @ @ Cmin$= @ @ Clist$APPEND @ @ Clist$APPEND VL1 @ @ Clist$CONS Ve Clist$NIL VL2 VL @ @ Cbool$/\\ @ Cbool$~ @ @ Cbool$IN Ve @ Clist$LIST_TO_SET VL1 @ Cbool$~ @ @ Cbool$IN Ve @ Clist$LIST_TO_SET VL2',
 '@

In [35]:
# compute the encoded fact pool at the beginning of each episode
#polished_goal = env.fringe["content"][0]["polished"]["goal"]

#use test_history for now
polished_goal = test_history[0]["content"][0]["polished"]["goal"]


# split_fact_pool = [i.strip().split() for i in fact_pool if i != polished_goal]
candidate_args = [t for t in fact_pool if t != polished_goal]
split_fact_pool = [i.strip().split() for i in candidate_args]


encoded_fact_pool = []

for fact in split_fact_pool:
    out, _ = batch_encoder.encode([fact])
    #output is bidirectional so concat vectors
    encoded_fact_pool.append(torch.cat(out.split(1), dim=2).squeeze(0))

# convert to jax
encoded_fact_pool = jnp.stack([jnp.array(encoded_fact_pool[i][0]) for i in range(len(encoded_fact_pool))])

#print (len(candidate_args), len(split_fact_pool))
#print (encoded_fact_pool.shape) #(len(candidates), 256)


In [36]:
# argument network for tactics with one or multiple arguments

   
hidden = jnp.expand_dims(target_representation,0)
# hidden = (hidden0, hidden1)

# concatenate the candidates with hidden states (i.e. goal).

goal_stack = jnp.concatenate([jnp.expand_dims(target_representation,0) for _ in encoded_fact_pool])

candidates = jnp.concatenate([encoded_fact_pool, goal_stack], 1)


init_state = hk.LSTMState(hidden, hidden)

#init_state
tac_idx
# hc = torch.cat([hidden0.squeeze(), hidden1.squeeze()])
# hiddenl = [hc.unsqueeze(0) for _ in split_fact_pool]

# hiddenl = torch.cat(hiddenl)

# # size: (len(fact_pool), 512)
# candidates = torch.cat([encoded_fact_pool, hiddenl], dim=1)
# candidates = candidates.to(device)

# input = tac_tensor

# # run it once before predicting the first argument
# hidden, _ = arg_net(input, candidates, hidden)

# # the indices of chosen args
# arg_step = []
# arg_step_probs = []
# for i in range(ARG_LEN):
#     hidden, scores = arg_net(input, candidates, hidden)
#     arg_probs = F.softmax(scores, dim=0)
#     arg_m = Categorical(arg_probs.squeeze())
#     arg = arg_m.sample()
#     arg_step.append(arg)
#     arg_step_probs.append(arg_m.log_prob(arg))

#     # hiddenc0 = hidden[0].squeeze().repeat(1, 1, 1)
#     # hiddenc1 = hidden[1].squeeze().repeat(1, 1, 1)

#     # encoded chosen argument
#     input = encoded_fact_pool[arg].unsqueeze(0).unsqueeze(0)
#     # print(input.shape)

#     # renew candidates                
#     hc = torch.cat([hidden0.squeeze(), hidden1.squeeze()])
#     hiddenl = [hc.unsqueeze(0) for _ in split_fact_pool]

#     hiddenl = torch.cat(hiddenl)

#     # size: (len(fact_pool), 512)
#     candidates = torch.cat([encoded_fact_pool, hiddenl], dim=1)
#     candidates = candidates.to(device)

# arg_pool.append(arg_step_probs)

# tac = tactic_pool[tac]
# arg = [candidate_args[i] for i in arg_step]

# tactic = env.assemble_tactic(tac, arg)

DeviceArray(4, dtype=int32)

In [37]:
init_arg, apply_arg = hk.transform(policy_networks._arg_forward)
arg_params = init_arg(rng_key, tac_idx, candidates, init_state, len(tactic_pool), 256)

apply_arg = partial(jax.jit, static_argnums=(5,6))(apply_arg)


In [38]:
#out_arg = apply_arg(arg_params,  rng_key, tac_idx, candidates, init_state, len(tactic_pool), 256)

#function for sampling single argument given previous arguments, 
def sample_arg(arg_params, arg_net, rng_key, input_, candidates, hidden, tactic_size, embedding_dim):
    hidden, arg_scores = arg_net(arg_params, rng_key, input_, candidates, hidden, tactic_size, embedding_dim)
    arg_scores = jnp.ravel(arg_scores)
    arg_probs = jax.nn.softmax(arg_scores)
    arg_idx = random.categorical(rng_key, arg_probs)
    log_prob = jnp.log(arg_probs[arg_idx])
    return log_prob, (arg_idx, hidden)

In [39]:
#print (init_state)
# run through first with tac_idx to initialise state with tactic as c_0
_, (_, hidden) = jax.grad(sample_arg, has_aux=True)(arg_params, apply_arg, rng_key, tac_idx, candidates, init_state, len(tactic_pool), 256)
#print (hidden)
# Q: Do we need to make the hidden state the goal again, now with updated cell from first pass? or is new hidden fine?

In [40]:
#log gradient of context policy for all arguments will be sum of log gradients of each argument.
#hence can just take the sum over gradients one at a time 

#should we add an argument terminating character or just let it run to arg_len?

gradients = []

ARG_LEN = 5
input_ = tac_idx
for _ in range(ARG_LEN):
    grad_log_arg, (arg_idx, hidden) = jax.grad(sample_arg, has_aux=True)(arg_params, apply_arg, rng_key, input_, candidates, hidden, len(tactic_pool), 256)
    gradients.append(grad_log_arg)
    input_ = jnp.expand_dims(encoded_fact_pool[arg_idx], 0)
    #renewing candidates sems redundant from old code?
    
gradients

[{'arg_policy/~/embed': {'embeddings': DeviceArray([[0., 0., 0., ..., 0., 0., 0.],
                [0., 0., 0., ..., 0., 0., 0.],
                [0., 0., 0., ..., 0., 0., 0.],
                ...,
                [0., 0., 0., ..., 0., 0., 0.],
                [0., 0., 0., ..., 0., 0., 0.],
                [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)},
  'arg_policy/~/linear': {'b': DeviceArray([-3.15498412e-02, -3.88397928e-03,  1.65330525e-03,
                 1.16532212e-02,  0.00000000e+00, -1.51223456e-03,
                -1.30844684e-02, -7.49786431e-03, -6.58695062e-04,
                 1.27873500e-03,  2.22407654e-03,  1.13938227e-02,
                -1.63530058e-03, -4.57714341e-05,  4.51708026e-03,
                 6.03670394e-03,  1.07955933e-02,  1.60729159e-02,
                -7.48921942e-04,  1.30142681e-02,  2.74061710e-02,
                -7.26708211e-04,  0.00000000e+00, -9.02514160e-03,
                 5.25332568e-03, -4.89337556e-03,  1.35844271e-03,
            

In [41]:
#can multiply learning rate by reward for each update to give the policy gradient after grad of log probs is done 
# context_lr = 1e-2
# # tactic_lr = 1e-2 
# # arg_lr = 1e-2
# # term_lr = 1e-2

# context_optimiser = optax.rmsprop(context_lr)
# # tactic_optimiser = optax.rmsprop(tactic_lr)
# # arg_optimiser = optax.rmsprop(arg_lr)
# # term_optimiser = optax.rmsprop(term_lr)

# opt_state_context = context_optimiser.init(context_params)
# # opt_state_tactic = tactic_optimiser.init(tactic_params)
# # opt_state_arg = arg_optimiser.init(arg_params)
# # opt_state_term = term_optimiser.init(term_params)





#gradient example. May need to construct separate function for each net during training 

# def compute_probs(params, net, *args):
#     probs = jax.nn.softmax(jnp.ravel(net(params,*args)))
#     logits = jnp.log(probs)
#     ind = random.categorical(rng_key, logits)
#     log_prob = logits[ind]
#     return log_prob

# grad = jax.grad(compute_probs)(context_params, apply_context, rng_key, jax_reps)#c_term, x_arg,TAC_SIZE, MAX_LEN)

# # updates, opt_state_term = context_optimiser.update(grad, opt_state_context)
# context_params = optax.apply_updates(context_params, updates)


# context_params
